# Cluster analysis
In this tutorial, we will perform the entire `scMGCA` cluster analysis using the Qx Bladder dataset dataset (can be downloaded <a href="https://drive.google.com/drive/folders/1BIZxZNbouPtGf_cyu7vM44G5EcbxECeu">here</a>).

## Import python package

In [4]:
import os
import argparse
import pandas as pd
import tensorflow as tf
from spektral.layers import GraphConv
from sklearn import metrics
from numpy.random import seed
seed(1)
tf.random.set_seed(1)
# Remove warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
# scMGCA module
from scMGCA.preprocess import *
from scMGCA.utils import *
from scMGCA.scmgca import SCMGCA, SCMGCAL
from scMGCA.losses import *
from scMGCA.graph_function import *

## Parameter settings

In [13]:
parser = argparse.ArgumentParser(description="train", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument("--dataname", default = "Quake_10x_Bladder", type = str)
parser.add_argument("--highly_genes", default = 500, type=int)
parser.add_argument("--pretrain_epochs", default = 1000, type=int)
parser.add_argument("--maxiter", default = 300, type=int)
args = parser.parse_args(args=['--dataname', 'Quake_10x_Bladder', '--highly_genes', '500', '--pretrain_epochs', '1000', '--maxiter', '300'])

## Read data

In [14]:
data = './dataset/'+args.dataname+'/data.h5'
x, y = prepro(data)
cluster_number = len(np.unique(y))
print("Cell number:", x.shape[0])
print("Gene number",x.shape[1])        
print("Cluster number:", cluster_number)

Cell number: 2500
Gene number 23341
Cluster number: 4


## Data preprocessing

In [15]:
x = np.ceil(x).astype(np.int)
adata = sc.AnnData(x)
adata.obs['Group'] = y
adata = normalize(adata, copy=True, highly_genes=args.highly_genes, size_factors=True, normalize_input=True, logtrans_input=True)

## Construct cell graph

In [ ]:
count = adata.X
adj = get_adj(count)
adj_n = GraphConv.preprocess(adj)

## Pre-training

In [17]:
model = SCMGCA(count, adj=adj, adj_n=adj_n)
model.pre_train(epochs=args.pretrain_epochs)
latent_pre = model.embedding(count, adj_n)

Epoch 10  Mult_loss: 0.0029011532   A_rec_loss: 0.31470138
Epoch 20  Mult_loss: -0.015144596   A_rec_loss: 0.25908032
Epoch 30  Mult_loss: -0.031327844   A_rec_loss: 0.15848605
Epoch 40  Mult_loss: -0.051763695   A_rec_loss: 0.033741966
Epoch 50  Mult_loss: -0.08169778   A_rec_loss: 0.022088172
Epoch 60  Mult_loss: -0.1273609   A_rec_loss: 0.01647526
Epoch 70  Mult_loss: -0.20676383   A_rec_loss: 0.014043791
Epoch 80  Mult_loss: -0.37768197   A_rec_loss: 0.013985698
Epoch 90  Mult_loss: -0.78664976   A_rec_loss: 0.01526031
Epoch 100  Mult_loss: -1.5655493   A_rec_loss: 0.016171148
Epoch 110  Mult_loss: -1.9093715   A_rec_loss: 0.014860538
Epoch 120  Mult_loss: -2.0988972   A_rec_loss: 0.013071854
Epoch 130  Mult_loss: -2.1883917   A_rec_loss: 0.012399688
Epoch 140  Mult_loss: -2.2425332   A_rec_loss: 0.012110037
Epoch 150  Mult_loss: -2.2792387   A_rec_loss: 0.011897248
Epoch 160  Mult_loss: -2.3088083   A_rec_loss: 0.01175571
Epoch 170  Mult_loss: -2.3341951   A_rec_loss: 0.011648619


## Training

In [18]:
centers = init_center(args, latent_pre, adj_n, cluster_number)
Cluster_predicted=model.train(epochs=args.maxiter, centers=centers)

Epoch 0  Mult_loss:  -2.6977754  A_rec_loss:  0.009536077  cluster_loss:  0.14840695
Epoch 10  Mult_loss:  -2.5016093  A_rec_loss:  0.009309375  cluster_loss:  0.14243658
Epoch 20  Mult_loss:  -2.5999277  A_rec_loss:  0.009281411  cluster_loss:  0.13151222
Epoch 30  Mult_loss:  -2.6170561  A_rec_loss:  0.0092894565  cluster_loss:  0.11304958
Epoch 40  Mult_loss:  -2.6233497  A_rec_loss:  0.009289366  cluster_loss:  0.09988786
Epoch 50  Mult_loss:  -2.6266727  A_rec_loss:  0.009287632  cluster_loss:  0.09036012
Epoch 60  Mult_loss:  -2.6317556  A_rec_loss:  0.009252352  cluster_loss:  0.08314127
Epoch 70  Mult_loss:  -2.6360266  A_rec_loss:  0.009267595  cluster_loss:  0.077201374
Epoch 80  Mult_loss:  -2.6396744  A_rec_loss:  0.009255022  cluster_loss:  0.0722112
Epoch 90  Mult_loss:  -2.6436422  A_rec_loss:  0.009254993  cluster_loss:  0.06802508
Epoch 100  Mult_loss:  -2.647798  A_rec_loss:  0.009276437  cluster_loss:  0.06441607
Epoch 110  Mult_loss:  -2.652613  A_rec_loss:  0.00926

## Evaluation

In [19]:
y = list(map(int, y))
Cluster_predicted.y_pred = np.array(Cluster_predicted.y_pred)
nmi = metrics.normalized_mutual_info_score(y, Cluster_predicted.y_pred)
ari = metrics.adjusted_rand_score(y, Cluster_predicted.y_pred)
print('NMI= %.4f, ARI= %.4f' % (nmi, ari))

NMI= 0.9802, ARI= 0.9900
